In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *

In [10]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.memory", "4g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Config

In [11]:
environment = '' # dev;uat;prd

In [ ]:
base_path = '../../../data'  # This will be overwritten by Papermill

In [12]:
path = {
    'slv': {'order_items': f'{base_path}/silver/evt/evt_order_items'}, 
    'snk': f'{base_path}/gold/fct/fct_order_items'
}

## Load

In [13]:
df = spark.read.format('delta').load(path['slv']['order_items'])

In [14]:
df_snk = df.select(
    F.col("id_order"),
    F.col("nr_order_item"),
    F.col("id_product"),
    F.col("id_seller"),
    F.col("dh_shipping_limit"),
    F.col("vl_item"),
    F.col("vl_freight")
)

In [15]:
df_snk = df_snk.withColumn("dh_insert_gld", F.current_timestamp())

## Sink

In [16]:
df_snk.write.format('delta')\
.mode(saveMode='overwrite')\
.save(path['snk'])